In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from functools import partial
import random as rand

## MMD Functions

In [ ]:
def maximum_mean_discrepancy(source_samples, target_samples, minimum=0., unbiased=False):
    """ This Maximum Mean Discrepancy (MMD) loss is calculated with a number of different Gaussian kernels.

    """

    sigmas = [
        1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10, 15, 20, 25, 30, 35, 100,
        1e3, 1e4, 1e5, 1e6
    ]
    gaussian_kernel = partial(_gaussian_kernel_matrix, sigmas=sigmas)
    if unbiased:
        loss_value = _mmd_kernel_unbiased(source_samples, target_samples, kernel=gaussian_kernel)
    else:
        loss_value = _mmd_kernel(source_samples, target_samples, kernel=gaussian_kernel)
        
        
    loss_value = tf.maximum(minimum, loss_value) 
    return loss_value

def _gaussian_kernel_matrix(x, y, sigmas):
    """ Computes a Gaussian Radial Basis Kernel between the samples of x and y.

    We create a sum of multiple gaussian kernels each having a width :math:`\sigma_i`.

    Parameters
    ----------
    x :  tf.Tensor of shape (M, num_features)
    y :  tf.Tensor of shape (N, num_features)
    sigmas : list(float)
        List which denotes the widths of each of the gaussians in the kernel.

    Returns
    -------
    kernel: tf.Tensor
        RBF kernel of shape [num_samples{x}, num_samples{y}]
    """
    def norm(v):
        return tf.reduce_sum(tf.square(v), 1)
    beta = 1. / (2. * (tf.expand_dims(sigmas, 1)))
    dist = tf.transpose(norm(tf.expand_dims(x, 2) - tf.transpose(y)))
    s = tf.matmul(beta, tf.reshape(dist, (1, -1)))
    kernel = tf.reshape(tf.reduce_sum(tf.exp(-s), 0), tf.shape(dist))
    return kernel

def _mmd_kernel(x, y, kernel=_gaussian_kernel_matrix):
    """ Computes the Maximum Mean Discrepancy (MMD) of two samples: x and y.

    Maximum Mean Discrepancy (MMD) is a distance-measure between the samples of the distributions of x and y.

    Parameters
    ----------
    x      : tf.Tensor of shape (num_samples, num_features)
    y      : tf.Tensor of shape (num_samples, num_features)
    kernel : callable, default: _gaussian_kernel_matrix
        A function which computes the kernel in MMD.

    Returns
    -------
    loss : tf.Tensor
        squared maximum mean discrepancy loss, shape (,)
    """

    loss = tf.reduce_mean(kernel(x, x))  # lint error: sigmas unfilled
    loss += tf.reduce_mean(kernel(y, y))  # lint error: sigmas unfilled
    loss -= 2 * tf.reduce_mean(kernel(x, y))  # lint error: sigmas unfilled
    return loss

def _mmd_kernel_unbiased(x, y, kernel=_gaussian_kernel_matrix):
    """ Computes the Maximum Mean Discrepancy (MMD) of two samples: x and y.

    Maximum Mean Discrepancy (MMD) is a distance-measure between the samples of the distributions of x and y.

    Parameters
    ----------
    x      : tf.Tensor of shape (num_samples, num_features)
    y      : tf.Tensor of shape (num_samples, num_features)
    kernel : callable, default: _gaussian_kernel_matrix
        A function which computes the kernel in MMD.

    Returns
    -------
    loss : tf.Tensor
        squared maximum mean discrepancy loss, shape (,)
    """
    m, n = x.shape[0], y.shape[0]
    loss = (1.0/(m*(m+1))) * tf.reduce_sum(kernel(x, x))  # lint error: sigmas unfilled
    loss += (1.0/(n*(n+1))) * tf.reduce_sum(kernel(y, y))  # lint error: sigmas unfilled
    loss -= (2.0/(m*n)) * tf.reduce_sum(kernel(x, y))  # lint error: sigmas unfilled
    return loss

In [ ]:
def mmd_permutation(x, y, n_perm=1000, kernel=_gaussian_kernel_matrix, unbiased=False):
    """
    Computes the p-value of the MMD by permuting the samples.
    """
    
    # Obtain sample sizes
    m = int(x.shape[0])
    n = int(y.shape[0])
    xy = tf.concat((x, y), axis=0)
    
    # Prepare MMD
    sigmas = [
        1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10, 15, 20, 25, 30, 35, 100,
        1e3, 1e4, 1e5, 1e6
    ]
    gaussian_kernel = partial(_gaussian_kernel_matrix, sigmas=sigmas)
    mmd2_null = np.zeros(n_perm)
    
    # Run permutations
    for i in range(n_perm):
        idx = np.random.permutation(m+n)
        xy = tf.gather(xy, idx, axis=0)
        if unbiased:
            mmd2 = _mmd_kernel_unbiased(xy[:m], xy[m:], kernel=gaussian_kernel)
        else:
            mmd2 = _mmd_kernel(xy[:m], xy[m:], kernel=gaussian_kernel)
        mmd2_null[i] = mmd2
    return mmd2_null

## Compute MMD

In [ ]:
pred_rt_fast_dm = np.load("../saved_arrays/pred_rt_fast_dm.npy")
pred_rt_neural = np.load("../saved_arrays/gpddm_pred_rt_neural.npy")
emp_rt = np.load("../saved_arrays/emp_rt_joint.npy")

In [ ]:
# initialize data structures
n_sub = emp_rt.shape[0]
n_sim = pred_rt_neural.shape[1]
n_perm = 1000

mmd_dynamic_emp = np.empty((n_sub, n_sim), dtype=np.float32)
mmd_dynamic_emp_perm = np.empty((n_sub, n_sim, n_perm), dtype=np.float32)

mmd_dynamic_fast = np.empty((n_sub, n_sim), dtype=np.float32)
mmd_dynamic_fast_perm = np.empty((n_sub, n_sim, n_perm), dtype=np.float32)

mmd_fast_emp = np.empty(n_sub, dtype=np.float32)
mmd_fast_emp_perm = np.empty((n_sub, n_perm), dtype=np.float32)

In [ ]:
# iterate over persons
for sub in range(n_sub):
    # subset person empirical data and fast-dm prediction
    x_fast_dm = np.atleast_2d(pred_rt_fast_dm[sub]).T.astype(np.float32)
    x_emp = np.atleast_2d(emp_rt[sub]).T.astype(np.float32)

    # calculate mmd and permutation between emp and fast-dm
    mmd_fast_emp[sub] = maximum_mean_discrepancy(x_fast_dm, x_emp)
    # mmd_fast_emp_perm[i] = mmd_permutation(x_fast_dm, x_emp)

    for sim in range(n_sim):
        # subset prediction of dynamic model
        x_dynamic = np.atleast_2d(pred_rt_neural[sub, sim]).T.astype(np.float32)

        # calculate mmd and permutation between dynamic and emp
        mmd_dynamic_emp[sub, sim] = maximum_mean_discrepancy(x_dynamic, x_emp)
        # mmd_dynamic_emp_perm[i, j] = mmd_permutation(x_dynamic, x_emp)

        # calculate mmd and permutation between dynamic and fast-dm
        mmd_dynamic_fast[sub, sim] = maximum_mean_discrepancy(x_dynamic, x_fast_dm)
        # mmd_dynamic_fast_perm[i, j] = mmd_permutation(x_dynamic, x_fast_dm)
        
    print("Calculation for Subject {} has finished...".format(sub+1))

In [ ]:
mmd_dynamic_emp.mean()

In [ ]:
mmd_dynamic_emp.std()

In [ ]:
mmd_fast_emp.mean()

In [ ]:
mmd_fast_emp.std()

In [ ]:
mmd_dynamic_fast.mean()

In [ ]:
mmd_dynamic_fast.std()